# OpenAI Responses API

## What is the OpenAI Responses API?

The Responses API is a new API released in March 2025. It is a combination of the traditional 
Chat Completions API and the Assistants API, providing support for:

- **Traditional Chat Completions:** Facilitates seamless conversational AI experiences.
- **Web Search:** Enables real-time information retrieval from the internet.
- **File Search:** Allows searching within files for relevant data.

Accordingly, the Assistants API will be retired in 2026. 

> **For new users, OpenAI recommends using the Responses API instead of the Chat Completions API to leverage its expanded capabilities.**

For a comprehensive comparison between the Responses API and the Chat Completions API, refer to the official OpenAI documentation: 
[Responses vs. Chat Completions](https://platform.openai.com/docs/guides/responses-vs-chat-completions).

## Summary of This Notebook
This notebook provides a hands-on guide for using the **OpenAI Responses API** to analyze tweets. 
It covers essential techniques such as:

- **Creating a vector store** and uploading tweets for semantic search.
- **Using file search** to analyze private datasets.
- **Performing a web search** to retrieve the latest public information.
- **Utilizing stateful responses** to maintain conversation context.
- **Combining file and web search** to enhance retrieval-augmented generation (RAG) applications.

By the end of this notebook, users will be able to integrate OpenAI's Responses API for efficient data retrieval and analysis of structured and unstructured data.

## Install Required Libraries
To use the OpenAI Responses API, we need to install the following libraries:

- **`openai`**: Provides access to OpenAI's APIs, including the Responses API

In [1]:
pip install openai -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sparkmagic 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.3.3 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


## Import Required Libraries

In [2]:
from IPython.display import Markdown, display
import boto3
from botocore.exceptions import ClientError
import json
import io

## Retrieve Secrets from AWS Secrets Manager

In [3]:
def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    return json.loads(secret)

## Initialize OpenAI Client

In [4]:
from openai import OpenAI
openai_api_key  = get_secret('openai')['api_key']

client = OpenAI(api_key=openai_api_key)

## File Search API

### Introduction to File Search
File search API enables efficient retrieval of relevant information 
from uploaded files by leveraging vector-based indexing. This feature is particularly useful 
for searching large datasets, extracting insights, and improving retrieval-augmented generation (RAG) applications.

Unlike traditional keyword-based searches, the Responses API uses embeddings 
to identify semantically relevant content, making it ideal for analyzing structured 
and unstructured text data (OpenAI, 2025).

For more details, visit the official OpenAI documentation: 
[File Search in Responses API](https://platform.openai.com/docs/guides/tools-file-search).

### Create a Vector Store

In [5]:
vector_store = client.vector_stores.create(
    name="my_vector_store"
)
vector_store_id = vector_store.id
print(vector_store_id)

vs_69126466f59881919f59f67e36c8bca7


### Upload Files

In [6]:
with open('tweet_text.json', 'rb') as f:
    file = client.files.create(
        file=f,            # file-like object
        purpose="assistants"
    )

file_id = file.id
print(file_id)

file-TNCe9haP3bYuBr3zuH2boX


### Attach File to Vector Store

In [7]:
attach_status =client.vector_stores.files.create(
    vector_store_id=vector_store_id,
    file_id=file_id
            )

print(attach_status.id)

file-TNCe9haP3bYuBr3zuH2boX


### Query the Vector Store

In [8]:
query = "the latest development in generativeAI"

In [9]:
search_results = client.vector_stores.search(
    vector_store_id=vector_store_id,
    query=query
)

for result in search_results.data[:5]:
    print(result.content[0].text[:100] + '\n Relevant score: ' + str(result.score))

They have been VERY clear that Chatgpt, these AI videos…"
  }
},
{
  "_id": {
    "$oid": "68e56b1f1
 Relevant score: 0.6010846063542131
Learning Plans. Use coupon code 𝐒𝐏𝐋𝟑𝟎 at checkout.🎯\n\nJoin Now 👉 https://t.co/LS2JuCrVmz\n\n#AI #Ce
 Relevant score: 0.5987929345462765
Create stunning, cinematic videos from a prompt—now with audio, physics, and cameos. One prompt = en
 Relevant score: 0.5911156596667607
⚡\n\nThe global #GenerativeAI market will hit $1.18 B this year — act now or get left behind. 🚀\n\n🔗
 Relevant score: 0.5728063202962544
They have been VERY clear that Chatgpt, these AI videos…"
  }
},
{
  "_id": {
    "$oid": "68e56b1d1
 Relevant score: 0.5622353955032359


## OpenAI Response API

### Simple Response

In [10]:
simple_response = client.responses.create(
  model="gpt-4o",
  input=[
      {
          "role": "user",
          "content": query
      }
  ]
)

In [11]:
display(Markdown(simple_response.output_text))

As of the latest updates, there are several key developments in the field of generative AI:

1. **Large Language Models (LLMs):** Models like GPT-4 have become increasingly sophisticated, with improvements in understanding and generating human-like text. They're being used in various applications, from customer support to creative content generation.

2. **Multimodal Models:** AI systems that can understand and generate multiple forms of data, such as text, image, and audio, are gaining traction. These models can perform tasks like generating images from textual descriptions or generating captions for images.

3. **Efficient Training Techniques:** Researchers are focusing on making training more efficient, reducing the carbon footprint and computational resources required to develop large models.

4. **Ethical and Bias Mitigation:** There is ongoing work to address ethical concerns in AI, such as bias and fairness, with efforts to make models less biased and more equitable.

5. **Open Source Initiatives:** Several organizations are releasing open-source models and tools, democratizing access to generative AI technology.

6. **Real-Time Applications:** Generative AI is increasingly used in real-time applications, such as live translation, personalized education, and adaptive gaming environments.

7. **AI in Creativity and Arts:** Artists and creators are leveraging AI to produce innovative artworks, music, and literature, often in collaboration with human creativity.

These developments highlight the rapid evolution and diverse applications of generative AI, with ongoing research and innovation continuing to push the boundaries.

### File Search Response

In [12]:

file_search_response = client.responses.create(
    input= query,
    model="gpt-4o",
    temperature = 0,
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store_id],
    }]
)

In [13]:
display(Markdown(file_search_response.output_text))


The latest developments in generative AI include:

1. **Agentic Workflows**: Amazon Web Services is exploring the future of AI with a focus on agentic workflows, which are being showcased alongside startups like NeuralSeek and Tarpit AI.

2. **AI in Supply Chain**: Atos has developed an AI-powered Supply Chain Disruption Analysis using generative AI, SAP BTP, and AWS Bedrock to assess risk and boost resilience.

3. **Market Growth**: The global generative AI market is projected to reach $1.18 billion this year, highlighting the rapid growth and adoption of this technology.

4. **Creative Industries**: Generative AI is transforming creative industries by enabling the creation of cinematic videos from simple prompts, integrating audio, physics, and cameos.

5. **Enterprise Applications**: IBM's Watsonx is bringing generative AI to enterprises, allowing teams to build custom large language models for enhanced customer engagement and streamlined processes.

These developments indicate a broadening application of generative AI across various sectors, from creative content generation to enterprise solutions and supply chain management.

## Web Search API

### Introduction to Web Search
The OpenAI Web Search tool allows models to retrieve real-time information from the internet. 
This capability is particularly useful for obtaining up-to-date data, fact-checking, and expanding knowledge 
without relying solely on pre-trained information. 

By leveraging OpenAI's web search functionality, the Responses API can fetch external data 
and provide accurate, relevant results in real time (OpenAI, 2025). 
This feature enhances applications that require the latest insights, such as news aggregation, research, 
or dynamic content generation.

For more details, visit the official OpenAI documentation: 
[Web Search in Responses API](https://platform.openai.com/docs/guides/tools-web-search).

### Perform Web Search

In [14]:
web_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= query,
    tools=[
        {
            "type": "web_search"
        }
    ]
)

In [15]:
display(Markdown(web_search_response.output_text))

Below is a comprehensive, up-to-date overview of the *latest developments in generative AI* as of today, **Monday, November 10, 2025**, including developments through early November. This summary is structured into key thematic categories and provides clear citations throughout.

---

##  Major Product Launches and Model Releases

- **OpenAI releases Sora 2 and Sora app (September 30, 2025):**  
  OpenAI unveiled **Sora 2**, a powerful multimodal video+audio generation model, accompanied by the **Sora app**, which enables users to create short-form AI-generated clips and share within a social feed. The system features synchronized dialogue and multi-shot consistency, marking a significant leap in consumer-facing generative capabilities. ([champaignmagazine.com](https://champaignmagazine.com/2025/10/05/ai-by-ai-weekly-top-5-september-29-october-5-2025/?utm_source=openai))

- **OpenAI’s Agentic Commerce Protocol & Instant Checkout (September 29, 2025):**  
  With this update, ChatGPT can now perform end-to-end commerce tasks—searching partner inventories, selecting products, and completing purchases via “Instant Checkout,” fully within the chat interface. ([champaignmagazine.com](https://champaignmagazine.com/2025/10/05/ai-by-ai-weekly-top-5-september-29-october-5-2025/?utm_source=openai))

- **Google’s October AI announcements (posted November 4, 2025):**  
  - **Gemini Enterprise**: Introduced as the “front door” to workplace AI, enabling enterprises to build, manage, and govern AI agents tailored to company data. ([blog.google](https://blog.google/technology/ai/google-ai-updates-october-2025/?utm_source=openai))  
  - **Gemini 2.5 Computer Use model**: A developer-facing model capable of interacting directly with user interfaces—greatly enhancing automation tasks like form-filling. ([blog.google](https://blog.google/technology/ai/google-ai-updates-october-2025/?utm_source=openai))  
  - **Flow updates (Veo 3.1)**: Enhancements include multi-image control, audio generation, and cinematic storytelling via generative AI. ([blog.google](https://blog.google/technology/ai/google-ai-updates-october-2025/?utm_source=openai))  
  - **“Gemini for Home”**: Replaces Google Assistant on smart devices with a more conversational, context-aware interface. ([blog.google](https://blog.google/technology/ai/google-ai-updates-october-2025/?utm_source=openai))  
  - **AI for fusion energy and cancer research**: A new quantum algorithm, and an AI model called *Cell2Sentence‑Scale* aimed at accelerating cancer therapy research. ([blog.google](https://blog.google/technology/ai/google-ai-updates-october-2025/?utm_source=openai))

- **Anthropic’s Claude Haiku 4.5:**  
  This new version of the company’s compact language model matches the performance of its Sonnet 4 model but operates at a third of the cost and more than twice the speed. Available across free tier subscriptions. ([computerworld.com](https://www.computerworld.com/article/4073957/anthropic-releases-new-version-of-its-smaller-haiku-model.html?utm_source=openai))

---

##  Cutting-Edge Image, Video, and Multimodal Models

- **Google’s “Nano Banana” (Gemini 2.5 Flash Image):**  
  Launched in August 2025, Nano Banana became a viral sensation, enabling photorealistic “3D figurine” image editing—allowing users to alter hairstyles, backdrops, and mix photos intelligently. It also includes SynthID watermarking to mark AI-generated content. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Nano_Banana?utm_source=openai))

- **Runway's Gen‑4 for video generation:**  
  Released on March 31, 2025, Gen‑4 can generate 5–10 second video clips from text prompts and reference imagery at 720p resolution. It achieves improved character consistency, realistic motion, and stylized camera movements—though with limitations such as short duration and fixed frame rate. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Gen-4_%28AI_image_and_video_model%29?utm_source=openai))

---

##  Emerging Trends and Research Frontiers

- **Agentic AI and autonomous agents rising:**  
  AI systems are transitioning from assistants to autonomous agents capable of reasoning, planning, and executing tasks. Use cases now include marketing campaign optimization, workflow execution, and enterprise automation. ([simplilearn.com](https://www.simplilearn.com/top-technology-trends-and-jobs-article?utm_source=openai))

- **Hybrid reasoning, long-term memory, and multi-agent systems:**  
  This new era of AI combines fast responses with step-by-step logic and persistent memory across sessions. Multi-agent systems and hybrid architectures (e.g., neural-symbolic transformers) enable formal logic, semantic reasoning, and backtracking—pushing AI toward more robust and interpretable performance. ([champaignmagazine.com](https://champaignmagazine.com/2025/07/01/ai-by-ai-first-half-of-2025-themes-and-breakthroughs/?utm_source=openai))

- **AI advancements in science and biotech:**
  - *OpenCRISPR‑1*: The first-ever human genome-editing enzyme designed entirely by generative AI, developed by Profluent Bio—the result of training a protein language model and capable of generating enzymes highly divergent from known variants. ([champaignmagazine.com](https://champaignmagazine.com/2025/08/10/ai-by-ai-weekly-top-5-august-4-10-2025/?utm_source=openai))  
  - *Diffusion models in protein design*: Models like RFDiffusion outperform traditional methods in backbone and sequence generation across 25 protein design tasks—reducing cost and improving success rates. ([arxiv.org](https://arxiv.org/abs/2504.16479?utm_source=openai))  
  - *Quantum generative advantage*: A 68-qubit superconducting quantum processor demonstrating generative model capabilities that are provably beyond classical computing, enabling efficient learning and sampling. ([arxiv.org](https://arxiv.org/abs/2509.09033?utm_source=openai))

- **Semantic frameworks for multimedia communication:**  
  A new information-theoretic approach introduces semantic entropy, mutual information, and rate-distortion tailored to ensuring semantic fidelity in generative multimedia systems. ([arxiv.org](https://arxiv.org/abs/2508.17163?utm_source=openai))

---

##  Business, Regulation, and Industry Impact

- **NVIDIA invests $100 billion in OpenAI compute infrastructure:**  
  The deal secures OpenAI significant access to NVIDIA’s Vera Rubin data center chips as it scales its computing capacity. The first 1 GW deployment is scheduled for H2 2026. ([spglobal.com](https://www.spglobal.com/market-intelligence/en/news-insights/research/2025/10/generative-ai-digest-massive-funding-haul-for-top-ai-players?utm_source=openai))

- **Adobe’s AI agents for marketing and CX:**  
  Adobe launched enterprise-oriented AI agents embedded into its Experience Platform, offering capabilities like journey orchestration, experimentation, and site optimization. ([spglobal.com](https://www.spglobal.com/market-intelligence/en/news-insights/research/2025/10/generative-ai-digest-massive-funding-haul-for-top-ai-players?utm_source=openai))

- **China’s first AI regulation targeting generative services:**  
  The "Interim AI Measures" regulate generative AI services administratively in China, marking a regulatory milestone. ([whitecase.com](https://www.whitecase.com/insight-our-thinking/ai-watch-global-regulatory-tracker-china?utm_source=openai))

---

##  Summary – What to Watch in the Coming Months

With November 10, 2025 as our current date, the most recent high-impact developments include:

- OpenAI's **Sora 2** and **Agentic Commerce Protocol** (late September)  
- Google's October announcements: **Gemini Enterprise**, **Gemini 2.5 Computer Use model**, **Flow/Veo 3.1**, **Gemini for Home**, and breakthroughs in **fusion energy & cancer AI**  
- Anthropic’s **Claude 4.5** release  
- Regulatory momentum (e.g., **China’s Interim AI Measures**)  
- Large investments and strategic ties such as NVIDIA’s $100B backing of OpenAI

 Together, these developments illustrate how generative AI is expanding across creative media, enterprise automation, scientific discovery, regulation, and commercialization. Let me know if you’d like deeper coverage on any of these topics.


### Stateful Response

The OpenAI Responses API includes a stateful feature that enables continuity in interactions. 
By using the `response_id`, a conversation can persist across multiple queries, 
allowing users to refine or expand upon previous searches. This is particularly useful for iterative research, 
dynamic content generation, and applications that require follow-up queries based on prior responses.

In [16]:
fetched_response = client.responses.retrieve(response_id=web_search_response.id)
display(Markdown(fetched_response.output_text[:100]))

Below is a comprehensive, up-to-date overview of the *latest developments in generative AI* as of to

### Continue Query with Web Search

In [21]:
continue_query = 'find different news'

continue_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= continue_query,
    previous_response_id=web_search_response.id,
    tools=[
        {
            "type": "web_search"
        }
    ]
)

In [22]:
display(Markdown(continue_search_response.output_text))

Here are some **recent and distinct developments in generative AI** across various domains, based on credible news sources as of **November 10, 2025**. Each section includes in-depth coverage and multiple citations to ensure accuracy.

---

## TIME Unveils the **TIME AI Agent** (Published Today)

- **What**: TIME has launched an innovative **AI-powered platform**, the *TIME AI Agent*, in collaboration with Scale AI. It enhances user interaction with journalism via language understanding, voice synthesis, translation, and search features—all unified in a conversational interface.

- **Purpose**: The agent allows users to generate article summaries, create audio reports, translate content, and explore information interactively—all while upholding TIME’s standards of accuracy and transparency.

- **Significance**: Represents a new phase in interactive journalism, transforming how readers consume news. This rollout marks a milestone in blending trusted editorial content with generative technology.  
([time.com](https://time.com/7332572/the-story-behind-the-time-ai-agent/?utm_source=openai))

---

## Policy Shift: **EU Considers Softening AI Act** (Reported 3 Days Ago)

- **What**: In response to heavy lobbying from Big Tech and U.S. trade concerns, the European Commission is contemplating watering down key provisions of its **AI Act**, effective in August 2024.

- **Proposed Changes**:
  - A one-year grace period for companies to comply.
  - Delay in enforcement of transparency fines until 2027.
  - Centralizing enforcement under a new EU AI office.  
  - Aimed at balancing compliance burdens with global competitiveness.

- **Significance**: Signals Europe's strategic rebalancing between regulatory stringency and economic positioning amid U.S.–China tensions.  
([ft.com](https://www.ft.com/content/af6c6dbe-ce63-47cc-8923-8bce4007f6e1?utm_source=openai))

---

## Microsoft & NVIDIA Launch **Agentic Launchpad** in UK (November 4)

- **What**: Microsoft, alongside NVIDIA and WeTransact, launched the **Agentic Launchpad** program in the UK and Ireland to support startups building autonomous, agentic AI.

- **Offerings**:
  - Technical mentoring from Microsoft and NVIDIA.
  - Azure cloud credits and access to NVIDIA startup training.
  - Commercial support including promotion through Microsoft’s marketplace and media.
  
- **Significance**: As part of Microsoft’s $30 billion AI investment, this initiative accelerates innovation in autonomous AI and bolsters the UK's role as an AI innovation hub.  
([windowscentral.com](https://www.windowscentral.com/microsoft/microsoft-and-nvidia-launch-uk-hub-to-fuel-the-next-wave-of-autonomous-ai-startups?utm_source=openai))

---

## OpenAI’s **Sora** App Sees Explosive First-Day Downloads (Nov 6–7)

- **What**: OpenAI launched its **Sora** AI video generation app on Android on **November 6**, with nearly **470,000 downloads** in the first day—significantly outperforming its iOS debut (approx. 360,000 downloads).

- **Significance**: Demonstrates surging consumer interest in generative AI-powered video tools and highlights a promising growth trajectory in the short-form AI video app market.  
([agiyes.com](https://www.agiyes.com/ainews/ai-news-from-november-1-7-2025/?utm_source=openai))

---

## Google’s Latest Moves in Generative AI – Key Updates (Nov 1–7)

Several notable initiatives have surfaced from Google this week:

1. **DS STAR** – A multi-agent AI framework that converts ambiguous business queries into executable Python code, working with varied data types like CSVs, JSON, Markdown, and unstructured text.  
2. **Meta’s Vibes Expansion** – *Vibes*, Meta’s AI-generated short video platform, has expanded into Europe, delivering content that is entirely AI-generated for entertainment platforms similar to TikTok.  
3. **Gemini 3 Pro Preview on Vertex AI** – A preview version of *Gemini 3 Pro*, with an expected 1-million-token context window, appeared in Vertex AI, suggesting imminent release.  
4. **Deep Research for Gemini** – A new feature allowing Gemini AI to extract and synthesize information from Gmail, Drive, and Chat for comprehensive research reports.  
5. **AI Mode in Chrome** – Google added an AI mode button to Chrome on iOS and Android in the U.S., with global expansion planned to make AI-powered search more accessible.  
6. **Nano Banana 2 Teaser** – Google hinted at a successor to its viral “Nano Banana” image model—**Nano Banana 2 (GEMPIX2)**—expected to launch soon.  
7. **ClickUp 4.0 Released** – ClickUp unveiled version 4.0 with a redesigned UI and two AI agents, integrating tasks, documents, messaging, and enterprise search into a unified experience.  
([agiyes.com](https://www.agiyes.com/ainews/ai-news-from-november-1-7-2025/?utm_source=openai))

---

## Philanthropic Initiative: **$500M Humanity AI**, Launched (Nov 2)

- **What**: A coalition of ten U.S. foundations—including MacArthur, Ford, Mellon, Mozilla, and Omidyar—jointly launched **Humanity AI**, a **$500 million**, five-year initiative dedicated to promoting AI development that serves public values.

- **Focus Areas**: Democracy, education, culture, labor, and security. The aim is to empower civil society voices and counterbalance corporate influence in AI governance.

- **Significance**: a landmark philanthropic investment in shaping AI toward ethical, inclusive, and community-centered applications.  
([ai-news-highlights.com](https://ai-news-highlights.com/ai-news-november-2-2025/?utm_source=openai))

---

## Summary of Current Trends

Several themes emerge from these developments:

- **Media & Content**: TIME's AI Agent and OpenAI’s Sora underline the rising integration of generative AI into journalism and creative media.
- **Regulatory Tensions**: The EU’s potential softening of AI rules highlights growing friction between regulation and competitiveness.
- **Innovation Support**: Microsoft and NVIDIA’s UK Launchpad shows strategic startup acceleration in autonomous AI.
- **Milestones in AI Infrastructure**: Google’s multifaceted updates reflect focused improvements in enterprise, multimodal, and developer-facing generative capabilities.
- **Ethical Momentum**: The Humanity AI initiative showcases a growing recognition that AI needs strong philanthropic and societal oversight alongside corporate and governmental frameworks.

If you'd like deeper exploration into any item—whether it's TIME’s AI Agent, Google's AI ecosystem, policy implications, or philanthropic governance—just let me know!

### Combining File Search and Web Search

This is an example of using file search to analyze private data and web search to retrieve public or the latest data. 
The Responses API allows developers to integrate these tools to enhance retrieval-augmented generation (RAG) applications. 
By combining file search with web search, users can leverage structured internal knowledge while also retrieving real-time 
information from external sources, ensuring comprehensive and up-to-date responses. 

In [23]:
combined_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= query,
    temperature = 0,
    instructions="Retrieve the results from the file search first, and use the web search tool to expand the results with news resources",
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store_id],
    },
        {
            "type": "web_search"
        }
    ]
)

In [24]:
display(Markdown(combined_search_response.output_text))

Recent developments in generative AI include:

1. **Agentic Workflows**: Amazon Web Services is exploring the future of AI with a focus on agentic workflows, which are being showcased alongside startups like NeuralSeek and Tarpit AI.

2. **AI in Supply Chain**: Atos has developed an AI-powered Supply Chain Disruption Analysis tool using generative AI, SAP BTP, and AWS Bedrock to assess risk and enhance resilience.

3. **Market Growth**: The global generative AI market is projected to reach $1.18 billion this year, highlighting the rapid growth and adoption of these technologies.

4. **Creative Industries**: Generative AI is transforming creative industries by enabling the creation of content, designs, and strategies, thus reshaping business innovation.

5. **Enterprise Applications**: IBM's Watsonx is bringing generative AI to enterprises with tools for secure and responsible deployment, allowing for custom large language models to enhance customer engagement and streamline processes.

These developments illustrate the expanding role of generative AI across various sectors, from supply chain management to creative industries and enterprise solutions.

# 🧩 Try It Yourself: Two-Step RAG (Private Data + Combined Search)

## Step 1 — Upload & Create Vector Store
1. Upload a short text file (e.g., `my_notes.txt`) to your notebook instance.  
2. Create a **vector store** and **ingest** your uploaded file.  
3. Run a simple test query to verify retrieval:  

In [56]:
vector_store = client.vector_stores.create(
    name="vector_store"
)
vector_store_id = vector_store.id
print(vector_store_id)

vs_69126c98b2588191b6bd99affafeffc4


In [57]:
with open('DOD.txt', 'rb') as f:
    file = client.files.create(
        file=f,            # file-like object
        purpose="assistants"
    )

file_id = file.id
print(file_id)

file-TFuTe7rRGRGxHA4cUxhaNn


In [58]:
attach_status =client.vector_stores.files.create(
    vector_store_id=vector_store_id,
    file_id=file_id
            )

print(attach_status.id)

file-TFuTe7rRGRGxHA4cUxhaNn


In [68]:
query = "summarize the main points from this article"

In [65]:
search_results = client.vector_stores.search(
    vector_store_id=vector_store_id,
    query=query
)

for result in search_results.data[:5]:
    print(result.content[0].text[:100] + '\n Relevant score: ' + str(result.score))

MEMORANDUM FOR SENIOR PENTAGON LEADERSHIP COMMANDERS OF CO MBA TANT COMMANDS DEFENSE AGENCY AND DOW 
 Relevant score: 0.8007805416154379
The Do W has initiated policy revisions to require building exportability into system design at the 
 Relevant score: 0.6237447904812116


## Step 2 — Combine File Search with Web Search
1. Enable both **file_search** and **web_search** in the Responses API.  
2. Use a prompt that asks the model to merge insights from both sources.  
   > Example: “Using my uploaded notes and the latest web information, summarize the current trends on this topic.”  
3. Review how the answer from your file and **current info** from the web.

✅ You’ve created a RAG system that combines **private** and **public** data for comprehensive, up-to-date analysis.


In [69]:
combined_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= query,
    temperature = 0,
    instructions="Retrieve the results from the file search first, and use the web search tool to expand the results with news resources",
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store_id],
    },
        {
            "type": "web_search"
        }
    ]
)

In [70]:
display(Markdown(combined_search_response.output_text))

The article discusses the Department of Defense's (DoD) efforts to enhance the efficiency and effectiveness of arms transfers and security cooperation. Key points include:

1. **Global Security and Partnerships**: The DoD emphasizes leveraging international partnerships to deliver capabilities to the U.S. military and allies, enhancing deterrence, and expanding the defense industrial base.

2. **Arms Transfers and Sales**: The U.S. maintains a high volume of Foreign Military Sales (FMS) and Direct Commercial Sales (DCS), with efforts to improve transparency and efficiency in these processes.

3. **Policy Revisions**: The DoD is revising policies to incorporate exportability into system designs early in the acquisition process and is investing in these features.

4. **Modernization and Transparency**: Plans are in place to modernize FMS and DCS IT systems to improve process performance and transparency, integrating various data to better forecast global demand.

5. **Organizational Realignment**: The Defense Security Cooperation Agency (DSCA) and the Defense Technology Security Administration (DTSA) are being realigned to optimize arms transfer efficiency and responsiveness.

6. **Strategic Alignment**: The realignment aims to ensure that arms transfer activities align with U.S. national security objectives and the National Defense Strategy.

7. **Industrial Base and Readiness**: The initiatives are expected to revitalize the industrial base, create jobs, and maintain the U.S. technological edge in defense capabilities.